In [58]:
import pandas as pd
import numpy as np
from math import trunc
import matplotlib.pyplot as plt
%matplotlib qt
import argparse, sys, os
import pygeodesy
from scipy.signal import butter, filtfilt, resample
from scipy.spatial.distance import cdist
from scipy.interpolate import make_interp_spline, make_smoothing_spline, BSpline
from scipy.ndimage import uniform_filter1d, spline_filter1d
from scipy.ndimage import gaussian_filter1d, uniform_filter1d


In [59]:
# current_path = os.path.dirname(__file__)

#put your relative path to the csv files
raptor_msgs = pd.read_csv('../data/pp_run4_2_raptor.csv')
imu_msgs = pd.read_csv('../data/pp_run4_2_imu.csv')
# raptor_msgs = pd.read_csv('../data/lvms_brake_test_raptor_msgs.csv')
# imu_msgs = pd.read_csv('../data/lvms_brake_test_imu_inputs.csv')
print(list(raptor_msgs.columns))
print(list(imu_msgs.columns))

def normalize_data(input_times, data):
    input_times = input_times[~np.isnan(data)]
    data = data[~np.isnan(data)]
    return np.column_stack([input_times, data])

raptor_times = np.array(raptor_msgs['__time'])
imu_times = np.array(imu_msgs['__time'])

raptor_times = np.round(raptor_times - imu_times[0], 3)
imu_times = np.round(imu_times - imu_times[0], 3)

KPH2MPH = 0.621371
rear_right_wheel_speed = np.array(raptor_msgs['/raptor_dbw_interface/wheel_speed_report/rear_right'])
rear_left_wheel_speed = np.array(raptor_msgs['/raptor_dbw_interface/wheel_speed_report/rear_left'])

rear_avg_wheel_speed = (rear_right_wheel_speed + rear_left_wheel_speed) / 2
rear_avg_wheel_speed = normalize_data(raptor_times, rear_avg_wheel_speed * KPH2MPH)
# desired_vel = normalize_data(imu_times, desired_vel)

filtered_rear_avg_wheel_speed = np.array(rear_avg_wheel_speed)
filtered_rear_avg_wheel_speed[:, 1] = gaussian_filter1d(rear_avg_wheel_speed[:, 1], 5)

['__time', '/raptor_dbw_interface/accelerator_pedal_report/control_type/value', '/raptor_dbw_interface/accelerator_pedal_report/driver_activity', '/raptor_dbw_interface/accelerator_pedal_report/enabled', '/raptor_dbw_interface/accelerator_pedal_report/fault_accel_pedal_system', '/raptor_dbw_interface/accelerator_pedal_report/fault_ch1', '/raptor_dbw_interface/accelerator_pedal_report/fault_ch2', '/raptor_dbw_interface/accelerator_pedal_report/header/stamp/nanosec', '/raptor_dbw_interface/accelerator_pedal_report/header/stamp/sec', '/raptor_dbw_interface/accelerator_pedal_report/ignore_driver', '/raptor_dbw_interface/accelerator_pedal_report/pedal_input', '/raptor_dbw_interface/accelerator_pedal_report/pedal_output', '/raptor_dbw_interface/accelerator_pedal_report/rolling_counter', '/raptor_dbw_interface/accelerator_pedal_report/torque_actual', '/raptor_dbw_interface/brake_2_report/estimated_road_slope', '/raptor_dbw_interface/brake_2_report/front_brake_pressure', '/raptor_dbw_interface

In [3]:
#Wheel speed with residuals
thr = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/accelerator_pedal_report/pedal_output']))
desired_vel = normalize_data(imu_times, np.array(imu_msgs['/long_control/set_desired_velocity/data']))
steering = normalize_data(imu_times, np.array(imu_msgs['/raptor_dbw_interface/steering_report/steering_wheel_angle']))

f2, axs = plt.subplots(2, 1, sharex=True)
f2.set_figwidth(15)
f2.set_figheight(10)
idxs = np.where(np.logical_and(rear_avg_wheel_speed[:, 0]>= 1200, rear_avg_wheel_speed[:, 0]<= 1380))
axs[0].scatter(rear_avg_wheel_speed[:, 0][idxs], rear_avg_wheel_speed[:, 1][idxs], s=.5, alpha=0.5, c='orange', label="Rear Avg Velocity")
axs[0].plot(filtered_rear_avg_wheel_speed[:, 0][idxs], filtered_rear_avg_wheel_speed[:, 1][idxs], 'b', label="Filtered Velocity")
axs[0].plot(desired_vel[:, 0][idxs], desired_vel[:, 1][idxs], 'r', label="Desired Velocity")
axs[0].legend()
axs[0].set_ylabel("Speed (MPH)")

axs[1].plot(filtered_rear_avg_wheel_speed[:, 0][idxs], desired_vel[:, 1][idxs] - filtered_rear_avg_wheel_speed[:, 1][idxs], 'b', linewidth = 0.5, label="Residual Velocity")
ax2 = axs[1].twinx()
ax2.plot(steering[:, 0][idxs], steering[:, 1][idxs] / 15, 'r', linewidth = 0.5, label="Steering Angle")
ax2.set_ylabel("Steering Angle (deg)")
ax2.set_ylim(-10, 10)

ax2.legend(loc="upper right")
axs[1].legend(loc="upper left")
axs[1].set_ylabel("Speed (MPH)")
# axs[1].set_xlabel("Time (Seconds)")
axs[1].set_ylim(-20, 20)


for ax in axs:
    ax.tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True)


# plt.savefig("/mnt/c/Users/johnl/OneDrive/College/!Current Semester/CS Research/putnam/velocity_with_steering.pdf", format='pdf', dpi=20000)

# print(len(filtered_rear_avg_wheel_speed[4:-2, 0]))
# print(len(desired_vel[:, 0]))
    
plt.show()

In [60]:
rear_brake_pressure = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/brake_2_report/rear_brake_pressure']))
front_brake_pressure = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/brake_2_report/front_brake_pressure']))
# des_brake_pressure = normalize_data(imu_times, np.array(imu_msgs['/joystick/brake_cmd/data']))
des_brake_pressure = normalize_data(imu_times, np.array(imu_msgs['/long_control/brake_cmd/data']))
accel_x = normalize_data(imu_times, np.array(imu_msgs['/accel/filtered/accel/accel/linear/x']))
nov_accel_x = normalize_data(imu_times, np.array(imu_msgs['/novatel_bottom/rawimu/linear_acceleration/y']))
v_accel_x = normalize_data(imu_times, np.array(imu_msgs['/vectornav/raw/attitude/linearaccelbody/x']))


brake_torque = (0.0050222 * 0.45 * 0.133 * (gaussian_filter1d(front_brake_pressure[:, 1], 1) * 1000) * 2) + (0.0050222 * 0.45 * 0.133 * (gaussian_filter1d(rear_brake_pressure[:, 1], 1) * 1000) * 2 )
brake_torque = brake_torque / (-1 * (0.3113 * 750))
req_brake_torque = (0.0050222 * 0.45 * 0.133 * (gaussian_filter1d(des_brake_pressure[:, 1], 1)) * 4)/ (-1 * (0.3113 * 750))

#Deceleration vs Brake Pressure
idxs = np.where(np.logical_and(rear_brake_pressure[:, 0]>= 1390, rear_brake_pressure[:, 0]<= 1430))
idx2 = np.where(np.logical_and(nov_accel_x[:, 0]>= 1390, nov_accel_x[:, 0]<= 1430))
f2, axs = plt.subplots(2, 1, sharex=True)
f2.set_figwidth(10)
f2.set_figheight(5)
axs[0].plot(rear_brake_pressure[:, 0][idxs], rear_brake_pressure[:, 1][idxs] / 1000, 'b', linewidth=0.5, label="Rear Brake Pressure")
axs[0].plot(front_brake_pressure[:, 0][idxs], front_brake_pressure[:, 1][idxs] / 1000, 'r', linewidth=0.5, label="Front Brake Pressure")
axs[0].plot(des_brake_pressure[:, 0][idxs], des_brake_pressure[:, 1][idxs] / 1000000, 'g', linewidth=0.5, label="Desired Brake Pressure")
# ax.plot(front_brake_pressure[:, 0], front_brake_pressure[:, 1] / ( rear_brake_pressure[:, 1] + front_brake_pressure[:, 1]) , 'g', label="")
axs[0].legend()
axs[0].set_ylabel("Pressure (Mpa)")
# axs[0].set_xlabel("Time (Seconds)")

axs[1].plot(accel_x[:, 0][idxs], accel_x[:, 1][idxs], 'b', linewidth=0.5, label="Linear Accel, X")
axs[1].plot(nov_accel_x[:, 0][idx2], gaussian_filter1d(-nov_accel_x[:, 1][idx2], 10), 'purple', linewidth=0.5, label="Nov Filtered Accel, X")
axs[1].plot(des_brake_pressure[:, 0][idxs], brake_torque[idxs], 'k', linewidth=0.5, label="Calculated Accel, X")
axs[1].plot(des_brake_pressure[:, 0][idxs], req_brake_torque[idxs], 'g', linewidth=0.5, label="Calc Req Accel, X")
axs[1].plot(v_accel_x[:, 0][idxs], uniform_filter1d(v_accel_x[:, 1][idxs],100, mode='constant', origin=(-100//2)), 'r', linewidth=0.5, label="Vectornav Accel, X")
axs[1].legend()
axs[1].set_ylabel("Acceleration (m/s^2)")
axs[1].set_xlabel("Time (Seconds)")

for ax in axs:
    ax.tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True)

# axs[1].set_ylim(15, -15)
# plt.savefig("/mnt/c/Users/johnl/OneDrive/College/!Current Semester/CS Research/putnam/putnam_brake_accel.pdf", format='pdf', dpi=20000)

plt.show()

NameError: name 'des_brake_pressure' is not defined

In [ ]:

thr = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/accelerator_pedal_report/pedal_output']))
gear = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/gear']))

#Deceleration vs Brake Pressure
idxs = np.where(np.logical_and(rear_brake_pressure[:, 0]>= 960, rear_brake_pressure[:, 0]<= 980))

f2, axs = plt.subplots(2, 1, sharex=True)
f2.set_figwidth(10)
f2.set_figheight(5)
MPH2MS = 0.44704
axs[0].scatter(thr[:, 0][idxs], thr[:, 1][idxs], s=.5, alpha=.5, c='orange', label="Unfiltered Throttle")
axs[0].plot(filtered_rear_avg_wheel_speed[:, 0][idxs], filtered_rear_avg_wheel_speed[:, 1][idxs] * MPH2MS, 'orange', label="Velocity")
axs[0].plot(thr[:, 0][idxs], gaussian_filter1d(thr[:, 1][idxs], 5), 'b', label="Throttle")
axs[0].set_ylim(0, 100)
axs[0].legend()
axs[0].set_ylabel("% and m/s")
# axs[0].set_xlabel("Time (Seconds)")

axs[1].plot(gear[:, 0][idxs], gear[:, 1][idxs], 'r', label="Gear")
axs[1].legend()
axs[1].set_ylabel("Gear")
axs[1].set_xlabel("Time (Seconds)")
axs[1].set_ylim(0, 6)

for ax in axs:
    ax.tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True)

# plt.savefig("/mnt/c/Users/johnl/OneDrive/College/!Current Semester/CS Research/lvms_brake_throttle_percent.pdf", format='pdf', dpi=20000)


plt.show()

In [71]:
engine_torque = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/torque_wheels']))
front_brake_pressure = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/brake_2_report/front_brake_pressure']))
rear_brake_pressure = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/brake_2_report/rear_brake_pressure']))
# des_brake_pressure = normalize_data(imu_times, np.array(imu_msgs['/long_control/brake_cmd/data']))
accel_x = normalize_data(imu_times, np.array(imu_msgs['/accel/filtered/accel/accel/linear/x']))
v_accel_x = normalize_data(imu_times, np.array(imu_msgs['/vectornav/raw/attitude/linearaccelbody/x']))
v_pitch = normalize_data(imu_times, np.array(imu_msgs['/vectornav/raw/attitude/yawpitchroll/y']))
nov = normalize_data(imu_times, np.array(imu_msgs["/novatel_bottom/rawimu/linear_acceleration/y"]))
gear = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/gear']))
e_eff = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/engine_efficiency']))
boost = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/boost_pressure']))
boost_target = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/boost_aim']))

print(list(v_pitch[1200:1250, 1]))

rpm = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/engine_speed']))
throttle = normalize_data(raptor_times, np.array(raptor_msgs['/raptor_dbw_interface/motec_report/throttle_position']))


torque_matrix = np.loadtxt('dyno_matrix_maker/new_data/output_matrix.csv', delimiter=',')
int_thr = np.array([int(x) for x in throttle[:, 1]])
int_rpms = np.array([int(x) for x in rpm[:,1]])
int_gears = np.array([int(x) for x in gear[:,1]])
gear_ratios = np.array([0, 2.9167, 1.875, 1.3809, 1.1154, 0.96, 0.8889])
final_drive = 3.0
torques = []
for item in range(len(int_thr)):
    if int_rpms[item] < 1600:
        torques.append(0)
    else:
        # torques.append(torque_matrix[int_rpms[item] - 1600][int_thr[item]])
        torques.append(torque_matrix[int_rpms[item] - 1600][int_thr[item]] * final_drive * gear_ratios[int_gears[item]])

calc_torque = np.column_stack([rpm[:,0], torques])

accels = np.array(torques)
calc_accel = np.column_stack([rpm[:,0], accels])

brake_torque = (0.0050222 * 0.48 * 0.133 * (gaussian_filter1d(front_brake_pressure[:, 1], 1) * 1000) * 2) + (0.0050222 * 0.48 * 0.133 * (gaussian_filter1d(rear_brake_pressure[:, 1], 1) * 1000) * 2 )
rolling_res = np.ones_like(brake_torque) * 9.81 * 0.01 * 4
mass = 750 + (0.5 * (rear_avg_wheel_speed[:, 1] * 0.44704)**2 * 2.27 * 0.15 * 1.22)
print(list(mass))
drag = (0.5 * (rear_avg_wheel_speed[:, 1] * 0.44704)**2 * 2.27 * 0.26 * 1.22)

# print(torque_matrix[5000-1600][35])
#Deceleration vs Brake Pressure
start = 10
# stop = engine_torque[-1, 0] - 10
stop = 1380
idxs = np.where(np.logical_and(engine_torque[:, 0]>= start, engine_torque[:, 0]<= stop))
idx2 = np.where(np.logical_and(v_accel_x[:, 0]>= start, v_accel_x[:, 0]<= stop))
idx3 = np.where(np.logical_and(nov[:, 0]>= start, nov[:, 0]<= stop))
f2, axs = plt.subplots(2, 1, sharex=True)
f2.set_figwidth(10)
f2.set_figheight(5)
# axs[0].plot(engine_torque[:, 0][idxs], engine_torque[:, 1][idxs], 'b', linewidth=0.5, label="Actual Torque")
# axs[0].plot(calc_torque[:, 0][idx2], calc_torque[:, 1][idx2], 'r', linewidth=0.5, label="Calculated Torque")
# axs[0].plot(accel_x[:, 0][idxs], accel_x[:, 1][idxs], 'b', linewidth=0.5, label="EKF Accel")
# axs[0].plot(v_accel_x[:, 0][idx2], gaussian_filter1d(v_accel_x[:, 1][idx2],20), 'g', linewidth=0.5, label="Vec Accel")
axs[0].plot(nov[:, 0][idx3], gaussian_filter1d(nov[:, 1][idx3] * -1,5), 'k', linewidth=0.5, label="Novatel Accel")
axs[0].plot(calc_torque[:, 0][idxs], (calc_torque[:, 1][idxs] - drag[idxs] - gaussian_filter1d(brake_torque[idxs], 10)) / (0.3113 * mass[idxs]) - (rolling_res[idxs] * np.cos(np.deg2rad(v_pitch[:, 1]))[idxs]) - (9.81 * np.sin(np.deg2rad(v_pitch[:, 1]))[idxs]), 'r', linewidth=0.5, label="Calculated Accel")
# axs[0].plot(des_brake_pressure[:, 0][idxs], des_brake_pressure[:, 1][idxs] / 1000000, 'g', linewidth=0.5, label="Desired Brake Pressure")
# ax.plot(front_brake_pressure[:, 0], front_brake_pressure[:, 1] / ( rear_brake_pressure[:, 1] + front_brake_pressure[:, 1]) , 'g', label="")
axs[0].legend()
axs[0].set_ylabel("Torque nm")
# axs[0].set_xlabel("Time (Seconds)")
print(e_eff[:, 1])
axs[1].plot(rear_avg_wheel_speed[:, 0][idxs], - (rolling_res[idxs] * np.cos(np.deg2rad(v_pitch[:, 1]))[idxs]) - (9.81 * np.sin(np.deg2rad(v_pitch[:, 1]))[idxs]), 'b', linewidth=0.5, label="Rolling and force")
axs[1].plot(v_pitch[:, 0][idxs], v_pitch[:, 1][idxs], 'r', linewidth=0.5, label="Rolling")
# ax2 = axs[1].twinx()
# ax2.plot(rpm[:, 0][idxs], rpm[:, 1][idxs], 'r', linewidth=0.5, label="RPM")
# ax2.set_ylabel("RPM")
# ax2.legend()
# axs[1].plot(v_accel_x[:, 0][idxs], gaussian_filter1d(v_accel_x[:, 1][idxs],10), 'r', linewidth=0.5, label="Vectornav Accel, X")
axs[1].legend()
axs[1].set_ylabel("Throttle %")
axs[1].set_xlabel("Time (Seconds)")

for ax in axs:
    ax.tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True)

# axs[1].set_ylim(15, -15)
# plt.savefig("/mnt/c/Users/johnl/OneDrive/College/!Current Semester/CS Research/lvms/torque_at_120.pdf", format='pdf', dpi=20000)

plt.show()

[1.319415569, 1.319334626, 1.311668277, 1.304009914, 1.30085206, 1.301608443, 1.298327923, 1.296735525, 1.299366593, 1.301814318, 1.299066424, 1.294876933, 1.287564993, 1.277156353, 1.26412034, 1.260868907, 1.251401782, 1.242996812, 1.239963055, 1.241182327, 1.246310949, 1.252544045, 1.258728743, 1.27267909, 1.285873175, 1.296494842, 1.299926281, 1.291074753, 1.289086461, 1.275774598, 1.270523071, 1.260660172, 1.254491329, 1.256681085, 1.25492394, 1.262618184, 1.263513446, 1.2628299, 1.268172145, 1.268449783, 1.271137118, 1.264512539, 1.258617401, 1.258178353, 1.256152987, 1.255427837, 1.251317143, 1.252957106, 1.257985234, 1.262411833]
[769.8576122776338, 769.744944774484, 769.576545590685, 769.6325978059889, 769.6325978059889, 769.5205713721485, 769.2974801084321, 769.4088654729629, 769.2974822300428, 769.2974822300428, 769.241908685939, 769.3531337861996, 769.3531337861996, 769.4646794211149, 769.4646794211149, 769.4088654729629, 769.4646794211149, 769.6325978059889, 769.68873015189